In [170]:
import time as tm
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
from selenium.webdriver.common.keys import Keys
import pyautogui

# Set Chrome options
chrome_options = webdriver.ChromeOptions()
# chrome_options.add_argument('--headless=new')   # Running Chrome in headless mode
chrome_options.add_argument('--no-sandbox')    # Necessary for running in some environments

# Automatically download and use the correct version of chromedriver
service = Service(ChromeDriverManager().install())

# Initialize the WebDriver with the service and options
driver = webdriver.Chrome(service=service, options=chrome_options)

In [171]:
def find_link():
    url = f'https://dienthoaivui.com.vn/may-cu/dien-thoai-cu'
    driver.get(url)

    # Đợi một khoảng thời gian để trang đã tải đầy đủ
    tm.sleep(3)

    while True:
        for i in range(0,10):
            pyautogui.scroll(-600)
            try:
                button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '//div[@class="w-full flex justify-center my-[10px]"]/button[1]')))
                button.click()
                tm.sleep(3)  
                print("Click 'Xem thêm' DONE")
                break
            except:
                if i == 9:
                    lst_xpath = '//div[@class="w-full grid grid-cols-2 w700:grid-cols-3 w1000:grid-cols-4 w1200:grid-cols-5 gap-[10px]"]/div/a'
                    tags = driver.find_elements(By.XPATH, lst_xpath)
                    print(f"Số lượng thẻ tìm thấy: {len(tags)}")
                    new_tags = [new_tag.get_attribute('href') for new_tag in tags]
                    return new_tags

In [172]:
def save_link_to_file(new_tags):
    import pickle
    # Lưu vào file
    with open('product_links_dienthoaivui.pkl', 'wb') as file:
        pickle.dump(new_tags, file)
def load_file_link_to_file(file_name = 'product_links_dienthoaivui.pkl'):
    import pickle
    with open(file_name, 'rb') as file:
        loaded_links = pickle.load(file)
    return loaded_links

In [173]:
def save_to_csv(data, feature_names):
    df = pd.DataFrame(data, columns=feature_names)
    df.to_csv("products_Dienthoaivui.csv", index=False, encoding='utf-8-sig')

In [174]:
def find_title():
    title_element = driver.find_element(By.TAG_NAME, 'h1')
    title = title_element.text.strip()
    return title

In [175]:
def find_price():
    try:
        sale_price_element = driver.find_element(By.XPATH, "//p[@class='inline-block font-bold text-dtv text-base w500:text-lg']")
        return sale_price_element.text.strip()
    except Exception as e:
        print(e)

In [176]:
def click_detail_button():
    for i in range(10):
        pyautogui.scroll(-500)
        try:
            guarantee = driver.find_element(By.XPATH, "//div[@class='w-full flex justify-center']/button")
            guarantee.click()
            print('Click xem cấu hình DONEEEEEEEEEEEEEEEEEEEEEEEEEEE!')
            return 1
        except:
            pass
    return 0

In [177]:
def find_hardcore():
    try:
        path = '//div[@class="w-full modal-technical-info-cps"]/div/table/tr'
        hardcore = driver.find_elements(By.XPATH, path)
        print("Len hardcore: ", len(hardcore))
    except:
        pass
    details = []
    titles = []
    for i in hardcore:
        lines = i.text.split("\n")
        chinh = lines[0] 
        phu = ' '.join(lines[1:])
        titles.append(chinh)
        details.append(phu)
    return details, titles


In [178]:
def prepare_data(data_raw):
    data = {"Tên sản phẩm": data_raw[0], "Giá bán cũ": data_raw[1], "Bảo Hành": data_raw[2]}
    details = data_raw[3]
    titles = data_raw[4]
    for title, detail in zip(titles, details):
        data[title] = detail
    return data

In [179]:
def guarantee():
    try:
        guarantee = driver.find_element(By.XPATH, "//div[@class='flex items-center justify-center text-ellipsis']/span")
        return guarantee.text.strip()
    except:
        return ""

In [180]:
# new_tags = find_link()
# save_link_to_file(new_tags)

In [181]:
if __name__ == "__main__":
    new_tags = load_file_link_to_file()
        # Lặp qua từng URL và tải trang
    data = []
    feature_names = ["Tên sản phẩm", "Giá bán cũ", "Bảo Hành"]
    count = 0
    for tag_url in new_tags:
        # if count == 10:
        #     break
        print('Link count: ', count)
        try:
            # Điều hướng đến từng URL sản phẩm
            driver.get(tag_url)
            tm.sleep(3)  # Đợi một khoảng thời gian ngắn để trang tải hoàn toàn
            title = find_title()
            sale_price_element = find_price()
            guarantee_text = guarantee()
            print(f"Tiêu đề sản phẩm: {title}")
            print(sale_price_element)
            print(guarantee_text)
            if click_detail_button() == 0:
                print("Thiếu cấu hình!")
                continue
            details, titles = find_hardcore()
            print(titles)
            print(details)
            data_raw = [title, sale_price_element, guarantee_text, details, titles]
            data_done = prepare_data(data_raw)

            for title in titles:
                if title not in feature_names:
                    feature_names.append(title)
            data.append(data_done)
            print("******************************************************************************")
            count += 1
        except Exception as e:
            count += 1
            print(f"Đã xảy ra lỗi khi lấy thông tin từ {tag_url}: {str(e)}")
            continue
    save_to_csv(data, feature_names)
    driver.quit()


Link count:  0
Tiêu đề sản phẩm: iPhone 13 Pro Max 256GB - Cũ đẹp
18.390.000 ₫
Bảo hành: 06 tháng
Click xem cấu hình DONEEEEEEEEEEEEEEEEEEEEEEEEEEE!
Len hardcore:  32
['Kích thước màn hình', 'Công nghệ màn hình', 'Độ phân giải màn hình', 'Tính năng màn hình', 'Tần số quét', 'Kiểu màn hình', 'Camera sau', 'Quay video', 'Tính năng camera', 'Camera trước', 'Chipset', 'GPU', 'Dung lượng RAM', 'Bộ nhớ trong', 'Pin', 'Công nghệ sạc', 'Cổng sạc', 'Thẻ SIM', 'Hệ điều hành', 'Jack tai nghe 3.5', 'Công nghệ NFC', 'Hỗ trợ mạng', 'Wi-Fi', 'Bluetooth', 'GPS', 'Kích thước', 'Trọng lượng', 'Chất liệu mặt lưng', 'Chất liệu khung viền', 'Chỉ số kháng nước, bụi', 'Các loại cảm biến', 'Tính năng đặc biệt']
['6.7 inches', 'OLED', '2778 x 1284 pixel', '120Hz, Super Retina XDR với ProMotion 6.1‑inch, OLED, 458 pp, HDR display, True Tone, Wide color (P3), Haptic Touch', '120Hz', 'Tai thỏ', 'Camera góc rộng: 12MP, ƒ/1.5 Camera góc siêu rộng: 12MP, ƒ/1.8 Camera tele : 12MP, /2.8', '4K @24 fps/25 fps/30 fps/ 60